# 用 Python 產生 Excel 圖表

我們今天來解決一個自動化 Excel 時經常遇到的問題：透過程式化的方式自動產生 Excel 圖表

在完成之後，我們會利用所學打造一個股票即時走勢圖的功能

在嘗試自動化圖表之前，我們先來觀察一下要如何透過手動的方式產生圖表

透過 Python 產生即時走勢圖的實作方法有很多種，本課程將會使用 Excel 内建的圖表功能來實作走勢圖

Excel 圖表的類型衆多，我們會使用 **開盤-高-低-收盤股價圖** 來繪製走勢圖：

1. 開啓 `TW2884.xlsx` 範例檔案
2. 接下來，請選擇工作表上 B 欄到 E 欄所有的資料，也就是玉山銀行一天的開-高-低-收四個時間序列

![](https://drive.google.com/uc?export=download&id=1Vd7lDwSCRpHG5-0JwEz3qwWfpRL1Dlwp)

3. 選擇好之後按下**插入 > 股票圖 > 開盤-高-低-收盤股價圖**

![](https://drive.google.com/uc?export=download&id=1XEkpufakOqFKHynuLpXENCcWOnaaUKng)

這樣就可以讓 Excel 幫我們把走勢圖產生出來了！


透過剛才的範例，我們注意到了，在繪製圖表前，最麻煩的事情是我們需要：

1. 先獲得繪製走勢圖的資料(開-高-低-收盤價)
2. 一筆筆交易日的資料，並且要把資料整理成以下格式：
![](https://drive.google.com/uc?export=download&id=1xWghyvrJQ5ZPjR4QN7Mz3_Zw5ujCVo4t)

幸好上述都可以透過 Python 自動化，我們先從資料的部分開始：



## 透過 fugle API 截取即時走勢資料

我們會使用之前介紹的截取我們需要的資料 

請先連至：[fugle API 文件](https://developer.fugle.tw/realtime/document#/Intraday/get_intraday_chart)

我們來看一下 `/intraday/chart` 這個 API

# 觀察一下回傳的結果

```json
{
  {'2019-06-28T01:01:00.000Z': {'close': 26.3,
    'high': 26.3,
    'low': 26.3,
    'open': 26.3,
    'unit': 426,
    'volume': 426000},
   '2019-06-28T01:02:00.000Z': {'close': 26.3,
    'high': 26.3,
    'low': 26.3,
    'open': 26.3,
    'unit': 47,
    'volume': 47000}
   }
}
```

我們要的資料是被放在 data > chart 内，**每一個交易時段與相對應的開盤-高-低-收盤股價是以 key : value 的形式被封裝在字典内**

In [5]:
import requests

payload = {
    "symbolId": "2884",
    "apiToken": api_token
}

res = requests.get("https://api.fugle.tw/realtime/v0/intraday/chart", params=payload)

result = res.json()
result

{'apiVersion': '0.1.0',
 'data': {'info': {'countryCode': 'TW',
   'date': '2019-11-12',
   'lastUpdatedAt': '2019-11-12T05:30:00.000Z',
   'mode': 'twse-sem',
   'symbolId': '2884',
   'timeZone': 'Asia/Taipei'},
  'chart': {'2019-11-12T01:01:00.000Z': {'close': 27.3,
    'high': 27.3,
    'low': 27.25,
    'open': 27.3,
    'unit': 793,
    'volume': 793000},
   '2019-11-12T01:02:00.000Z': {'close': 27.35,
    'high': 27.35,
    'low': 27.3,
    'open': 27.3,
    'unit': 150,
    'volume': 150000},
   '2019-11-12T01:03:00.000Z': {'close': 27.4,
    'high': 27.4,
    'low': 27.3,
    'open': 27.35,
    'unit': 250,
    'volume': 250000},
   '2019-11-12T01:04:00.000Z': {'close': 27.4,
    'high': 27.4,
    'low': 27.4,
    'open': 27.4,
    'unit': 60,
    'volume': 60000},
   '2019-11-12T01:05:00.000Z': {'close': 27.4,
    'high': 27.4,
    'low': 27.35,
    'open': 27.4,
    'unit': 213,
    'volume': 213000},
   '2019-11-12T01:06:00.000Z': {'close': 27.45,
    'high': 27.45,
    'lo

若要截取某一個時間點的開高低收資料，我們需要輸入時間戳：

```python
result["data"]["chart"]['2019-11-11T01:03:00.000Z']
```

才能將資料從該字典中提取出來：

```json
{'close': 27.45,
    'high': 27.5,
    'low': 27.4,
    'open': 27.5,
    'unit': 202,
    'volume': 202000}
```

再回頭看看我們期望的 Excel 工作表格式：

![](https://drive.google.com/uc?export=download&id=1xWghyvrJQ5ZPjR4QN7Mz3_Zw5ujCVo4t)


## 回憶一下第一堂課的一個口號

要用 Python 與 xlwings 一次性的將多筆資料寫入一個 Excel 的範圍，就**必須將寫入的資料放入大小與儲存格範圍一樣的二維串列内**

現在我們需要把剛才從 fugle api 截取到的**資料轉成二維陣列**，才有可能將其**批次的寫入 Excel 工作表**：


```python
[
    ['2019-06-28T01:01:00.000Z', '26.3', '26.3', '26.3', '26.3','426', '426000'],
    ['2019-06-28T01:02:00.000Z', '26.3', '26.3', '26.3', '26.3', '47', '47000'],
    ['2019-06-28T01:03:00.000Z', '26.3', '26.35', '26.3', '26.35', '88', '88000']
]
```


我們先將 fugle API 回傳給我們的 JSON 内，所有的 key，也就是時間戳提取出來：

```python
result["data"]["chart"].keys()
```

結果會回傳：

```python
dict_keys(['2019-11-11T01:01:00.000Z', '2019-11-11T01:02:00.000Z', '2019-11-11T01:03:00.000Z', '2019-11-11T01:04:00.000Z', '2019-11-11T01:05:00.000Z', '2019-11-11T01:06:00.000Z', '2019-11-11T01:07:00.000Z', '2019-11-11T01:08:00.000Z', '2019-11-11T01:09:00.000Z', ...
```

In [15]:
result["data"]["chart"].keys()

dict_keys(['2019-11-11T01:01:00.000Z', '2019-11-11T01:02:00.000Z', '2019-11-11T01:03:00.000Z', '2019-11-11T01:04:00.000Z', '2019-11-11T01:05:00.000Z', '2019-11-11T01:06:00.000Z', '2019-11-11T01:07:00.000Z', '2019-11-11T01:08:00.000Z', '2019-11-11T01:09:00.000Z', '2019-11-11T01:10:00.000Z', '2019-11-11T01:11:00.000Z', '2019-11-11T01:12:00.000Z', '2019-11-11T01:13:00.000Z', '2019-11-11T01:14:00.000Z', '2019-11-11T01:15:00.000Z', '2019-11-11T01:16:00.000Z', '2019-11-11T01:17:00.000Z', '2019-11-11T01:18:00.000Z', '2019-11-11T01:19:00.000Z', '2019-11-11T01:20:00.000Z', '2019-11-11T01:21:00.000Z', '2019-11-11T01:22:00.000Z', '2019-11-11T01:23:00.000Z', '2019-11-11T01:24:00.000Z', '2019-11-11T01:25:00.000Z', '2019-11-11T01:26:00.000Z', '2019-11-11T01:27:00.000Z', '2019-11-11T01:28:00.000Z', '2019-11-11T01:29:00.000Z', '2019-11-11T01:30:00.000Z', '2019-11-11T01:31:00.000Z', '2019-11-11T01:32:00.000Z', '2019-11-11T01:33:00.000Z', '2019-11-11T01:34:00.000Z', '2019-11-11T01:35:00.000Z', '2019-11-

In [29]:
time_idx = result["data"]["chart"].keys()

for idx in time_idx:
    data = result["data"]["chart"][idx]
    print(data)

{'close': 27.6, 'high': 27.6, 'low': 27.55, 'open': 27.6, 'unit': 501, 'volume': 501000}
{'close': 27.5, 'high': 27.55, 'low': 27.5, 'open': 27.55, 'unit': 118, 'volume': 118000}
{'close': 27.55, 'high': 27.6, 'low': 27.5, 'open': 27.5, 'unit': 83, 'volume': 83000}
{'close': 27.45, 'high': 27.5, 'low': 27.45, 'open': 27.5, 'unit': 449, 'volume': 449000}
{'close': 27.45, 'high': 27.5, 'low': 27.4, 'open': 27.5, 'unit': 202, 'volume': 202000}
{'close': 27.4, 'high': 27.45, 'low': 27.4, 'open': 27.4, 'unit': 57, 'volume': 57000}
{'close': 27.4, 'high': 27.45, 'low': 27.4, 'open': 27.4, 'unit': 37, 'volume': 37000}
{'close': 27.4, 'high': 27.45, 'low': 27.4, 'open': 27.4, 'unit': 74, 'volume': 74000}
{'close': 27.35, 'high': 27.4, 'low': 27.35, 'open': 27.4, 'unit': 371, 'volume': 371000}
{'close': 27.35, 'high': 27.35, 'low': 27.35, 'open': 27.35, 'unit': 142, 'volume': 142000}
{'close': 27.35, 'high': 27.4, 'low': 27.3, 'open': 27.35, 'unit': 452, 'volume': 452000}
{'close': 27.3, 'high'

接下來我們就將資料放入一個二維陣列中：

```python
price_open = []
price_high = []
price_low = []
price_close = []
unit = []
volume = []

# 截取所有的時間戳
time_idx = result["data"]["chart"].keys()

# 將每一個時間點的六筆資料分別放入不同的串列内
for idx in time_idx:
    data = result["data"]["chart"][idx]
    price_open.append(data["open"])
    price_high.append(data["high"])
    price_low.append(data["low"])
    price_close.append(data["close"])
    unit.append(data["unit"])
    volume.append(data["volume"])
 
# 最後再將這六個串列放入一個串列内
chart_data = [list(time_idx), price_open,price_high,price_low,price_close,unit,volume]
```

In [6]:
price_open = []
price_high = []
price_low = []
price_close = []
unit = []
volume = []

# 截取所有的時間戳
time_idx = result["data"]["chart"].keys()

# 將每一個時間點的六筆資料分別放入不同的串列内
for idx in time_idx:
    data = result["data"]["chart"][idx]
    price_open.append(data["open"])
    price_high.append(data["high"])
    price_low.append(data["low"])
    price_close.append(data["close"])
    unit.append(data["unit"])
    volume.append(data["volume"])

# 最後再將這六個串列放入一個串列内
chart_data = [list(time_idx), price_open,price_high,price_low,price_close,unit,volume]
chart_data

[['2019-11-12T01:01:00.000Z',
  '2019-11-12T01:02:00.000Z',
  '2019-11-12T01:03:00.000Z',
  '2019-11-12T01:04:00.000Z',
  '2019-11-12T01:05:00.000Z',
  '2019-11-12T01:06:00.000Z',
  '2019-11-12T01:07:00.000Z',
  '2019-11-12T01:08:00.000Z',
  '2019-11-12T01:09:00.000Z',
  '2019-11-12T01:10:00.000Z',
  '2019-11-12T01:11:00.000Z',
  '2019-11-12T01:12:00.000Z',
  '2019-11-12T01:13:00.000Z',
  '2019-11-12T01:14:00.000Z',
  '2019-11-12T01:15:00.000Z',
  '2019-11-12T01:16:00.000Z',
  '2019-11-12T01:17:00.000Z',
  '2019-11-12T01:18:00.000Z',
  '2019-11-12T01:19:00.000Z',
  '2019-11-12T01:20:00.000Z',
  '2019-11-12T01:21:00.000Z',
  '2019-11-12T01:22:00.000Z',
  '2019-11-12T01:23:00.000Z',
  '2019-11-12T01:24:00.000Z',
  '2019-11-12T01:25:00.000Z',
  '2019-11-12T01:26:00.000Z',
  '2019-11-12T01:27:00.000Z',
  '2019-11-12T01:28:00.000Z',
  '2019-11-12T01:29:00.000Z',
  '2019-11-12T01:30:00.000Z',
  '2019-11-12T01:31:00.000Z',
  '2019-11-12T01:32:00.000Z',
  '2019-11-12T01:33:00.000Z',
  '2019-11

In [7]:
wb = xw.Book()

sheet = wb.sheets[0]

sheet.range("A1").value = chart_data

# 但是結果卻出乎我們意料之外

![](https://drive.google.com/uc?export=download&id=1Cg304rzu-OMWVsZzYMwmZxOsEvmNEo5T)

原因是我們的二維串列的維度不對，**今天要將資料寫入一個二維的 Excel 工作表範圍，就需要賦值一個維度大小一樣的二維串列**
zhuanzhi
![](https://drive.google.com/uc?export=download&id=1Bxulsz0NpTCYg86VWLBmgdirMeeG7MtE)

我們現在需要把二維串列做一個**轉置（Transpose）的處理，利用 Numpy 套件可以很快完成**

```python
import numpy as np

chart_data_ary = np.array(chart_data)
# 轉置只需要針對一個 Numpy Array 執行 .T 即可
chart_data_ary.T
```

接下來我們將資料寫入 Excel 工作表：

```python
sheet.range("A1").value = np.array(chart_data).T
```

經過轉置後的陣列，最後就會在 Excel 上呈現：

![](https://drive.google.com/uc?export=download&id=1vQsd6Mu_ZW9_GeBGXJp-8tBNIzL1ytf9)

In [4]:
# 將我們目前的程式碼整理成一個函數
import requests
import numpy as np
import time

def get_chart_data(stock_id, api_token):
    payload = {
        "symbolId": stock_id,
        "apiToken": api_token
    }

    res = requests.get("https://api.fugle.tw/realtime/v0/intraday/chart", params=payload)

    result = res.json()

    price_open = []
    price_high = []
    price_low = []
    price_close = []
    unit = []
    volume = []

    time_idx = result["data"]["chart"].keys()

    for idx in time_idx:
        data = result["data"]["chart"][idx]
        price_open.append(data["open"])
        price_high.append(data["high"])
        price_low.append(data["low"])
        price_close.append(data["close"])
        unit.append(data["unit"])
        volume.append(data["volume"])

    chart_data = [list(time_idx), price_open,price_high,price_low,price_close,unit,volume]
    chart_data_ary = np.array(chart_data).T
    
    return chart_data_ary

In [5]:
api_token = "f5d2cef7e50eb84a842b01d39c5d6a8b"

#api_token = "你的 fugle api 同行碼"
get_chart_data("0050", api_token)

array([['2019-11-12T01:01:00.000Z', '91.55', '91.8', ..., '91.8', '200',
        '200000'],
       ['2019-11-12T01:02:00.000Z', '91.95', '91.95', ..., '91.9', '32',
        '32000'],
       ['2019-11-12T01:03:00.000Z', '91.8', '91.85', ..., '91.85', '12',
        '12000'],
       ...,
       ['2019-11-12T05:23:00.000Z', '91.9', '91.9', ..., '91.9', '2',
        '2000'],
       ['2019-11-12T05:24:00.000Z', '91.95', '91.95', ..., '91.95', '10',
        '10000'],
       ['2019-11-12T05:25:00.000Z', '92', '92', ..., '91.9', '15',
        '15000']], dtype='<U24')

In [6]:
import xlwings xw

wb = xw.Book()
sheet = wb.sheets[0]

## xlwings 的 chart 物件

用 `sheet.charts.add()` 產生新的圖表

上面的程式碼可以解讀成：我要在工作表的圖表集合新增一個圖表

```python
# 在工作表新增一個空圖表物件
chart = sheet.charts.add()
# 這個圖標物件目前就像是一個空白的畫布
# 我們需要丟一些資料進去，它才能夠畫出圖來
chart
```

![](https://drive.google.com/uc?export=download&id=1oIrE9EK-QgyN4Vlhr87-nyOjaXkAQVLQ)


```python
last_cell = sheet.range("A1").end("down").end("right")
last_cell
# <Range [活頁簿1]工作表1!$G$264>
```

## chart.set_source_data()

設定圖表的資料

```python
chart.set_source_data(sheet.range("B1:E264"))
```
![](https://drive.google.com/uc?export=download&id=14z_gdJTn6LJnWcgI2t_McMiveD6vETro)

但是這依然不是我們要的圖表樣式...

In [19]:
chart.chart_type = "stock_ohlc"

## 如何查詢 chart_type

Excel 的圖表有非常多種，可以參考微軟官網的文件：[xlChartTypes 微軟官網](https://docs.microsoft.com/zh-tw/office/vba/api/Excel.XlChartType)

![](https://drive.google.com/uc?export=download&id=1dI2-htl5Enb7hsiix67QAnFKf0yLjDa0)

## 設定圖表位置

```python
# 圓餅圖最上方等於 E1 儲存格的上邊
chart.top = sheet.range("I1").top
# 圓餅圖最左方等於 E1 儲存格的左邊
chart.left = sheet.range("I1").left
```

![](https://drive.google.com/uc?export=download&id=18XmzHX7_z3eRBP4LQafefR8is6t3C4JM)

In [21]:
# 圓餅圖最上方等於 E1 儲存格的上邊
chart.top = sheet.range("I1").top
# 圓餅圖最左方等於 E1 儲存格的左邊
chart.left = sheet.range("I1").left

## 完成版程式碼

In [8]:
import xlwings as xw
import requests
import numpy as np
import time

def get_chart_data(stock_id, api_token):
    payload = {
        "symbolId": stock_id,
        "apiToken": api_token
    }

    res = requests.get("https://api.fugle.tw/realtime/v0/intraday/chart", params=payload)

    result = res.json()

    price_open = []
    price_high = []
    price_low = []
    price_close = []
    unit = []
    volume = []

    time_idx = result["data"]["chart"].keys()

    for idx in time_idx:
        data = result["data"]["chart"][idx]
        price_open.append(data["open"])
        price_high.append(data["high"])
        price_low.append(data["low"])
        price_close.append(data["close"])
        unit.append(data["unit"])
        volume.append(data["volume"])

    chart_data = [list(time_idx), price_open,price_high,price_low,price_close,unit,volume]
    chart_data_ary = np.array(chart_data).T
    
    return chart_data_ary


api_token = "你的 fugle api 同行碼"

wb = xw.Book()
sheet = wb.sheets[0]
# 呼叫 api 截取資料
data = get_chart_data("2884", api_token)
sheet.range("A1").value = data
# 增加一個 chart 
chart = sheet.charts.add()
last_row = sheet.range("E1").end("down").row
# 選擇 B2 到 G264 這個範圍，
chart_data = sheet.range(f"B2:G{last_row}").value
# 將圖表的資料設定成該範圍内的資料
chart.set_source_data(sheet.range("B1:E264"))
# 最後將圖表的類別設定成 開盤-高-低-收盤股價圖
chart.chart_type = "stock_ohlc"
# 圓餅圖最上方等於 E1 儲存格的上邊
chart.top = sheet.range("I1").top
# 圓餅圖最左方等於 E1 儲存格的左邊
chart.left = sheet.range("I1").left